## Check if we have dbatools docker images

In [1]:
docker images "dbatools/*"

## Verify if we have containers setup

In [2]:
docker ps -a

### If containers are not up and running, start them

In [3]:
# start container dockersql1
docker start dockersql1

# start container dockersql2 
docker start dockersql2

## Set variables values

In [1]:
$dbatools1 = "localhost,1433"
$dbatools2 = "localhost,14333"
$database = "Pubs"
$AG = "test-ag"
$secureString = ConvertTo-SecureString "dbatools.IO" -AsPlainText -Force
$cred = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList "sqladmin", $secureString

## Remove 'test-ag' from AG

In [4]:
# Remove Database from AG
Remove-DbaAgDatabase -SqlInstance $dbatools1 -SqlCredential $cred -AvailabilityGroup $AG -Database $database -Confirm:$false

# Remove Availability group
Remove-DbaAvailabilityGroup -SqlInstance $dbatools1, $dbatools2 -SqlCredential $cred -AvailabilityGroup $AG -Confirm:$false

# Remove Database from secondary instance
Remove-DbaDatabase -SqlInstance $dbatools2 -SqlCredential $cred -Database $database -Confirm:$false

# Grant permissions

In [18]:
$filePermissions = "D:\Presentations\SQLSaturday#926 - Lisboa 2019\Reset\02_GrantPermissions.sql"

$scriptWithoutGO = Get-Content $filePermissions -Raw | ForEach-Object { 
    $_ -replace '\bGO\b',' '
}

Invoke-DbaQuery -SqlInstance $dbatools1 -SqlCredential $cred -Query $scriptWithoutGO -Verbose

### Check Permissions

In [19]:
Get-DbaUserPermission -SqlInstance $dbatools1 -SqlCredential $cred

### Remove files from directories

In [1]:
Remove-Item -Path "D:\Presentations\SQLSaturday#926 - Lisboa 2019\Replicate Permissions\*" -Force

<hr>

## Reset for copy data demo

Remove `EmptyNorthwind` database from `localhost,14333` instance

In [4]:
Remove-DbaDatabase -SqlInstance $dbatools2 -SqlCredential $cred -Database EmptyNorthwind -Confirm:$false

Drop table `[dbo].[CopyOf_Order Details]`

In [0]:
Invoke-DbaQuery -SqlInstance $dbatools1 -SqlCredential $cred -Database Northwind -Query "DROP TABLE [dbo].[CopyOf_Order Details]"